### Creating the SP500 historical database
The goal of this notebook is to construct a SQL database with SP500 historical data from the past 5 years.<br>
We are going to use the IEX API to collect the data from the SP500 listed companies, make some data processing and save it to RDS SQL database on AWS.<br>
This approach was inspired on Kaggle S&P 500 stock data (https://www.kaggle.com/camnugent/sandp500) and part of the code was extracted from the script _getSandP.py_ found on the dataset folder of the challenge.

In [9]:
# import libraries
from datetime import datetime
from concurrent import futures

import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import os

To use IEX API (https://iexcloud.io/docs/api/) trough pandas_datareader, we need to sign up to the service and provide an API key. 


In [10]:
# Define my API key for IEX
os.environ["IEX_API_KEY"] = 'pk_c876f405f2e441cba4bd913e6d066a7f'

The list of S&P500 companies was also extracted from the mentioned Kaggle page.

In [11]:
sp500_list_code = ['MMM','ABT','ABBV','ACN','ATVI','AYI','ADBE','AMD','AAP','AES','AET',
'AMG','AFL','A','APD','AKAM','ALK','ALB','ARE','ALXN','ALGN','ALLE',
'AGN','ADS','LNT','ALL','GOOGL','GOOG','MO','AMZN','AEE','AAL','AEP',
'AXP','AIG','AMT','AWK','AMP','ABC','AME','AMGN','APH','APC','ADI','ANDV',
'ANSS','ANTM','AON','AOS','APA','AIV','AAPL','AMAT','APTV','ADM','ARNC',
'AJG','AIZ','T','ADSK','ADP','AZO','AVB','AVY','BHGE','BLL','BAC','BK',
'BAX','BBT','BDX','BRK.B','BBY','BIIB','BLK','HRB','BA','BWA','BXP','BSX',
'BHF','BMY','AVGO','BF.B','CHRW','CA','COG','CDNS','CPB','COF','CAH','CBOE',
'KMX','CCL','CAT','CBG','CBS','CELG','CNC','CNP','CTL','CERN','CF','SCHW',
'CHTR','CHK','CVX','CMG','CB','CHD','CI','XEC','CINF','CTAS','CSCO','C','CFG',
'CTXS','CLX','CME','CMS','KO','CTSH','CL','CMCSA','CMA','CAG','CXO','COP',
'ED','STZ','COO','GLW','COST','COTY','CCI','CSRA','CSX','CMI','CVS','DHI',
'DHR','DRI','DVA','DE','DAL','XRAY','DVN','DLR','DFS','DISCA','DISCK','DISH',
'DG','DLTR','D','DOV','DWDP','DPS','DTE','DRE','DUK','DXC','ETFC','EMN','ETN',
'EBAY','ECL','EIX','EW','EA','EMR','ETR','EVHC','EOG','EQT','EFX','EQIX','EQR',
'ESS','EL','ES','RE','EXC','EXPE','EXPD','ESRX','EXR','XOM','FFIV','FB','FAST',
'FRT','FDX','FIS','FITB','FE','FISV','FLIR','FLS','FLR','FMC','FL','F','FTV',
'FBHS','BEN','FCX','GPS','GRMN','IT','GD','GE','GGP','GIS','GM','GPC','GILD',
'GPN','GS','GT','GWW','HAL','HBI','HOG','HRS','HIG','HAS','HCA','HCP','HP','HSIC',
'HSY','HES','HPE','HLT','HOLX','HD','HON','HRL','HST','HPQ','HUM','HBAN','HII',
'IDXX','INFO','ITW','ILMN','IR','INTC','ICE','IBM','INCY','IP','IPG','IFF','INTU',
'ISRG','IVZ','IQV','IRM','JEC','JBHT','SJM','JNJ','JCI','JPM','JNPR','KSU','K','KEY',
'KMB','KIM','KMI','KLAC','KSS','KHC','KR','LB','LLL','LH','LRCX','LEG','LEN','LUK',
'LLY','LNC','LKQ','LMT','L','LOW','LYB','MTB','MAC','M','MRO','MPC','MAR','MMC','MLM',
'MAS','MA','MAT','MKC','MCD','MCK','MDT','MRK','MET','MTD','MGM','KORS','MCHP','MU',
'MSFT','MAA','MHK','TAP','MDLZ','MON','MNST','MCO','MS','MOS','MSI','MYL','NDAQ',
'NOV','NAVI','NTAP','NFLX','NWL','NFX','NEM','NWSA','NWS','NEE','NLSN','NKE','NI',
'NBL','JWN','NSC','NTRS','NOC','NCLH','NRG','NUE','NVDA','ORLY','OXY','OMC','OKE',
'ORCL','PCAR','PKG','PH','PDCO','PAYX','PYPL','PNR','PBCT','PEP','PKI','PRGO','PFE',
'PCG','PM','PSX','PNW','PXD','PNC','RL','PPG','PPL','PX','PCLN','PFG','PG','PGR',
'PLD','PRU','PEG','PSA','PHM','PVH','QRVO','PWR','QCOM','DGX','RRC','RJF','RTN','O',
'RHT','REG','REGN','RF','RSG','RMD','RHI','ROK','COL','ROP','ROST','RCL','CRM','SBAC',
'SCG','SLB','SNI','STX','SEE','SRE','SHW','SIG','SPG','SWKS','SLG','SNA','SO','LUV',
'SPGI','SWK','SBUX','STT','SRCL','SYK','STI','SYMC','SYF','SNPS','SYY','TROW','TPR',
'TGT','TEL','FTI','TXN','TXT','TMO','TIF','TWX','TJX','TMK','TSS','TSCO','TDG','TRV',
'TRIP','FOXA','FOX','TSN','UDR','ULTA','USB','UAA','UA','UNP','UAL','UNH','UPS','URI',
'UTX','UHS','UNM','VFC','VLO','VAR','VTR','VRSN','VRSK','VZ','VRTX','VIAB','V','VNO',
'VMC','WMT','WBA','DIS','WM','WAT','WEC','WFC','HCN','WDC','WU','WRK','WY','WHR','WMB',
'WLTW','WYN','WYNN','XEL','XRX','XLNX','XL','XYL','YUM','ZBH','ZION','ZTS']

Next, we are going to iterate through each company, get their stock data from the past 5 years and concatenate on a dataframe

In [14]:
end_time = datetime.now()
start_time = datetime(end_time.year - 5, end_time.month , end_time.day)
print(end_time)
print(start_time)


2019-10-31 11:18:28.333997
2014-10-31 00:00:00


In [15]:
stock_df = web.DataReader('MSFT','iex', start_time, end_time)

In [21]:
stock_df = web.DataReader('AMZN','iex', start_time, end_time)

In [16]:
stock_df

,open,high,low,close,volume
date,,,,,
2014-10-31,46.94,46.97,46.48,46.95,35849656
2014-11-03,46.89,47.46,46.73,47.44,23130397
2014-11-04,47.30,47.73,47.25,47.57,21530813
2014-11-05,47.80,47.90,47.26,47.86,22449594
2014-11-06,47.86,48.86,47.79,48.70,33037841
...,...,...,...,...,...
2019-10-24,139.39,140.42,138.67,139.94,37278399
2019-10-25,139.34,141.14,139.20,140.73,25959724
2019-10-28,144.40,145.67,143.51,144.19,35280137


In [17]:
sp500_df = pd.DataFrame()

In [18]:
sp500_df

""


In [19]:
sp500_df = pd.concat([sp500_df, stock_df])

In [22]:
sp500_df = pd.concat([sp500_df, stock_df])

In [23]:
sp500_df

,open,high,low,close,volume
date,,,,,
2014-10-31,46.94,46.97,46.48,46.95,35849656
2014-11-03,46.89,47.46,46.73,47.44,23130397
2014-11-04,47.30,47.73,47.25,47.57,21530813
2014-11-05,47.80,47.90,47.26,47.86,22449594
2014-11-06,47.86,48.86,47.79,48.70,33037841
...,...,...,...,...,...
2019-10-24,1771.09,1788.34,1760.27,1780.78,5204350
2019-10-25,1697.55,1764.21,1695.00,1761.33,9626402
2019-10-28,1748.06,1778.70,1742.50,1777.08,3708851


In [12]:
for company in sp500_list_code:

In [6]:
# Define the start and end date
end_data = datetime.now()
start_time = datetime(end_data.year - 5, end_data.month , end_data.day)
print(end_data)
print(start_time)

2019-10-31 11:16:51.141940
2014-10-31 00:00:00


In [7]:
stock_df = web.DataReader('MSFT','iex', start_time, end_data)

ValueError: The IEX Cloud API key must be provided either through the api_key variable or through the  environment variable IEX_API_KEY